In [48]:
%load_ext autoreload
%autoreload 2
from u import *
from expres.src.config import Config

import util

config = Config(Path('results').mk(),
    board_dim=20,
    n_win=5,
                
    res_basic_block=True,
    res_inplanes=[16, 32, 64, 128],
    res_num_blocks=[3, 4, 6, 3],
                
    c_puct=1,
    mcts_iterations=100,
    temp=1,
    move_temp_decay=25,

    lr=1e-2,
    train_epochs=50000,
    train_batch=1024,
    pred_batch=24,
    early_stop=False,
    epoch_model_update=5,
    epoch_model_save=50,
    max_save=20,

    max_mcts_queue=200000,
    time_reset_eval=60 * 60,
    min_num_states=1024,
    num_mcts_processes=32
).save(force=True)

config = Config(Path('results_debug').mk(),
    board_dim=20,
    n_win=5,
                
    res_basic_block=True,
    res_inplanes=[16, 32, 64, 128],
    res_num_blocks=[3, 4, 6, 3],
                
    c_puct=1,
    mcts_iterations=1600,
    temp=1,

    lr=1e-2,
    train_epochs=50000,
    train_batch=1024,
    pred_batch=24,
    early_stop=False,
    epoch_model_update=5,
    epoch_model_save=50,
    max_save=20,

    max_mcts_queue=200000,
    time_reset_eval=60 * 60,
    min_num_states=256,
    num_mcts_processes=32
).save(force=True)
# config = Config(path,
#     res_basic_block=True,
#     res_inplanes=[16],
#     res_num_blocks=[1],
#     n_win=5,
#     board_dim=20,
#     c_puct=1,
#     mcts_iterations=100,
#     temp=1,
# )
util.config = config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# epochs 0 - 300: mcts iterations = 100, no decay, lr=0.01

In [ ]:
from network import ResNetGomoku

net = ResNetGomoku(config).to(config.device)
inp = torch.zeros((1, 2, config.board_dim, config.board_dim)).to(config.device)
out = net(inp)

In [23]:
from mcts import *

def ev(state):
    v, p = from_torch(net(to_torch(np.expand_dims(state, axis=0), device=config.device)))
    return v, p[0]
net.evaluate = ev

inp = np.zeros((2, config.board_dim, config.board_dim), dtype=np.float32)
mcts = MCTS(inp, net)
states, policies, values, indices = mcts.run()